<a href="https://colab.research.google.com/github/stiflerGit/CAD-Mammography/blob/master/InceptionResNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')

work_dir = "./drive/My Drive/RESnet/"
if not os.path.exists(work_dir):
  os.makedirs(work_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

"""
This script goes along my blog post:
Keras InceptionResetV2 (https://jkjung-avt.github.io/keras-inceptionresnetv2/)
"""
import os

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

VERSION = '0.0.1'

DATASET_PATH  = '/content/drive/My Drive/CI_FinalProject/'
IMAGE_SIZE    = (224, 224)
NUM_CLASSES   = 2
BATCH_SIZE    = 8  # try reducing batch size or freeze more layers if your GPU runs out of memory
FREEZE_LAYERS = 2  # freeze the first this many layers for training
NUM_EPOCHS    = 20
WEIGHTS_FINAL = 'PT-CNN%s.h5' % VERSION

TRAIN_IMGS_FILE_NAME = 'train_img_%d.npy' % IMAGE_SIZE[0]
TRAIN_LABELS_FILE_NAME = 'train_lab.npy'
TEST_IMGS_FILE_NAME = 'public_test_image_%d.npy' % IMAGE_SIZE[0]
TEST_LABELS_FILE_NAME = 'public_test_label.npy'


"""
    DATA PRE-PROCESSING
    """

import numpy as np
from sklearn.model_selection import train_test_split

train_gray_imgs = np.load(os.path.join(DATASET_PATH, TRAIN_IMGS_FILE_NAME))
train_lbls = np.load(os.path.join(DATASET_PATH, TRAIN_LABELS_FILE_NAME))

# print(train_lbls.shape) # 2864 images (it is a quite small dataset)
# print(np.count_nonzero(train_lbls)) # 1546 
# 1546 images belong to class 1, this means 1318 images belong to class 0
# so our class distribution is good

# we have to equally distribute otherwise the error:
### Training and validation subsets have different number of classes after the 
### split. If your numpy arrays are sorted by the label, you might want to shuffle them.
# could be raised. This is due to the distribution of classes of the labels.
# for example after the split, the training labels set contains both class 1 and 2,
# and on the other hand validation labels set contains either class 1 or class 2.
# Even if the train data would cointain both the classes it's better to equally 
# distribute the two classes over the sets.
val_split = 0.2
X_train, X_val, y_train, y_val = train_test_split(train_gray_imgs, train_lbls,
                                                  test_size=val_split, 
                                                  stratify=train_lbls)

X = np.concatenate((X_train, X_val))
y = np.concatenate((y_train, y_val))

train_samples = len(X_val)
valid_samples = len(X_val)
del(X_train, X_val)
del(y_train, y_val)

# this is done to transform the gray imgs into rgb images
# X = X.astype('float32')/65536
X = np.stack((X,)*3, axis=-1)

del(train_gray_imgs)
del(train_lbls)

test_gray_imgs = np.load(os.path.join(DATASET_PATH, TEST_IMGS_FILE_NAME))
test_rgb_imgs = np.stack((test_gray_imgs,)*3, axis=-1)
test_lbls = np.load(os.path.join(DATASET_PATH, TEST_LABELS_FILE_NAME))

del(test_gray_imgs)

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=40, 
    zoom_range=0.2,
    rescale=1./65536, 
    horizontal_flip=True, vertical_flip=True, 
    validation_split=val_split)

train_batches = train_datagen.flow(X, y, batch_size=32, subset='training')

valid_batches = train_datagen.flow(X, y, batch_size=8, subset='validation')

test_datagen = ImageDataGenerator(rescale=1./65536)
test_batches = test_datagen.flow(test_rgb_imgs, test_lbls, batch_size=BATCH_SIZE)

# show class indices
# print('****************')
# for cls, idx in train_batches.class_indices.items():
#     print('Class #{} = {}'.format(idx, cls))
# print('****************')

Using TensorFlow backend.


In [3]:

# build our classifier model based on pre-trained InceptionResNetV2:
# 1. we don't include the top (fully connected) layers of InceptionResNetV2
# 2. we add a DropOut layer followed by a Dense (fully connected)
#    layer which generates softmax class score for each class
# 3. we compile the final model using an Adam optimizer, with a
#    low learning rate (since we are 'fine-tuning')
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))

x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True
    
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(net_final.summary())

Instructions for updating:
Colocations handled automatically by placer.
219062272/219055592 [==============================] - 3s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 111, 111, 32) 96          conv2d[0][0]                     
_____________________________________________________

In [0]:
# train the model
history = net_final.fit_generator(train_batches,
                        steps_per_epoch = train_samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# save trained weights
# net_final.save(WEIGHTS_FINAL)

Epoch 1/20
72/72 [==============================] - 166s 2s/step - loss: 0.7430 - acc: 0.5964 - val_loss: 0.5988 - val_acc: 0.6189
Epoch 2/20
72/72 [==============================] - 132s 2s/step - loss: 0.5656 - acc: 0.7243 - val_loss: 0.5620 - val_acc: 0.7203
Epoch 3/20
72/72 [==============================] - 130s 2s/step - loss: 0.4944 - acc: 0.7757 - val_loss: 0.4931 - val_acc: 0.7587
Epoch 4/20
72/72 [==============================] - 133s 2s/step - loss: 0.4257 - acc: 0.8093 - val_loss: 0.4073 - val_acc: 0.8269
Epoch 5/20
72/72 [==============================] - 131s 2s/step - loss: 0.3898 - acc: 0.8281 - val_loss: 0.3442 - val_acc: 0.8531
Epoch 6/20
72/72 [==============================] - 130s 2s/step - loss: 0.3628 - acc: 0.8490 - val_loss: 0.3328 - val_acc: 0.8689
Epoch 7/20
72/72 [==============================] - 129s 2s/step - loss: 0.3391 - acc: 0.8530 - val_loss: 0.3222 - val_acc: 0.8566
Epoch 8/20
72/72 [==============================] - 129s 2s/step - loss: 0.3322 - a